## บทนำสู่การวิเคราะห์การถดถอย - บทเรียนที่ 1

#### ทำความเข้าใจในมุมมอง

✅ มีวิธีการวิเคราะห์การถดถอยหลายประเภท และการเลือกใช้วิธีใดขึ้นอยู่กับคำตอบที่คุณต้องการ หากคุณต้องการทำนายความสูงที่เป็นไปได้ของบุคคลในช่วงอายุหนึ่ง คุณจะใช้ `linear regression` เพราะคุณกำลังมองหาค่า **ตัวเลข** หากคุณสนใจที่จะค้นหาว่าประเภทของอาหารควรถือว่าเป็นมังสวิรัติหรือไม่ คุณกำลังมองหาการ **จัดหมวดหมู่** ดังนั้นคุณจะใช้ `logistic regression` คุณจะได้เรียนรู้เพิ่มเติมเกี่ยวกับ logistic regression ในภายหลัง ลองคิดถึงคำถามบางข้อที่คุณสามารถถามจากข้อมูล และวิธีการใดที่เหมาะสมที่สุด

ในส่วนนี้ คุณจะได้ทำงานกับ [ชุดข้อมูลขนาดเล็กเกี่ยวกับโรคเบาหวาน](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) ลองจินตนาการว่าคุณต้องการทดสอบการรักษาสำหรับผู้ป่วยโรคเบาหวาน โมเดล Machine Learning อาจช่วยคุณระบุว่าผู้ป่วยคนใดจะตอบสนองต่อการรักษาได้ดีกว่า โดยอิงจากการผสมผสานของตัวแปรต่าง ๆ แม้แต่โมเดลการถดถอยที่พื้นฐานที่สุด เมื่อถูกนำเสนอในรูปแบบภาพ อาจแสดงข้อมูลเกี่ยวกับตัวแปรที่ช่วยให้คุณจัดการทดลองทางคลินิกในเชิงทฤษฎีได้อย่างมีประสิทธิภาพ

เมื่อกล่าวเช่นนั้น มาเริ่มต้นงานนี้กันเลย!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>ผลงานศิลปะโดย @allison_horst</figcaption>

<!--![ผลงานศิลปะโดย \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>ผลงานศิลปะโดย @allison_horst-->


## 1. การเตรียมเครื่องมือของเรา

สำหรับงานนี้ เราจะต้องใช้แพ็กเกจดังต่อไปนี้:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) คือ [ชุดของแพ็กเกจใน R](https://www.tidyverse.org/packages) ที่ออกแบบมาเพื่อทำให้การวิเคราะห์ข้อมูลรวดเร็วขึ้น ง่ายขึ้น และสนุกมากขึ้น!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) เป็นกรอบการทำงานที่เป็น [ชุดของแพ็กเกจ](https://www.tidymodels.org/packages/) สำหรับการสร้างโมเดลและการเรียนรู้ของเครื่อง

คุณสามารถติดตั้งแพ็กเกจเหล่านี้ได้ด้วยคำสั่ง:

`install.packages(c("tidyverse", "tidymodels"))`

สคริปต์ด้านล่างจะตรวจสอบว่าคุณมีแพ็กเกจที่จำเป็นสำหรับการทำโมดูลนี้หรือไม่ และจะติดตั้งให้ในกรณีที่บางแพ็กเกจขาดหายไป


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



ตอนนี้ มาโหลดแพ็กเกจที่ยอดเยี่ยมเหล่านี้และทำให้พร้อมใช้งานในเซสชัน R ปัจจุบันของเรา (นี่เป็นเพียงตัวอย่าง `pacman::p_load()` ได้ทำสิ่งนี้ให้คุณแล้ว)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. ชุดข้อมูลโรคเบาหวาน

ในแบบฝึกหัดนี้ เราจะนำทักษะการวิเคราะห์การถดถอยมาใช้โดยการทำนายบนชุดข้อมูลโรคเบาหวาน ชุดข้อมูล [โรคเบาหวาน](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) ประกอบด้วย `442 ตัวอย่าง` ของข้อมูลเกี่ยวกับโรคเบาหวาน โดยมีตัวแปรคุณลักษณะเชิงพยากรณ์ 10 ตัว ได้แก่ `อายุ`, `เพศ`, `ดัชนีมวลกาย`, `ความดันโลหิตเฉลี่ย`, และ `การวัดระดับเซรั่มในเลือดหกตัว` รวมถึงตัวแปรผลลัพธ์ `y`: การวัดเชิงปริมาณของการพัฒนาของโรคในหนึ่งปีหลังจากจุดเริ่มต้น

|จำนวนการสังเกตการณ์|442|
|----------------------|:---|
|จำนวนตัวแปรพยากรณ์|10 คอลัมน์แรกเป็นตัวแปรเชิงพยากรณ์เชิงตัวเลข|
|ผลลัพธ์/เป้าหมาย|คอลัมน์ที่ 11 เป็นการวัดเชิงปริมาณของการพัฒนาของโรคในหนึ่งปีหลังจากจุดเริ่มต้น|
|ข้อมูลตัวแปรพยากรณ์|- อายุเป็นปี
||- เพศ
||- bmi ดัชนีมวลกาย
||- bp ความดันโลหิตเฉลี่ย
||- s1 tc, คอเลสเตอรอลรวมในเซรั่ม
||- s2 ldl, ไลโปโปรตีนความหนาแน่นต่ำ
||- s3 hdl, ไลโปโปรตีนความหนาแน่นสูง
||- s4 tch, คอเลสเตอรอลรวม / HDL
||- s5 ltg, อาจเป็นค่าลอการิทึมของระดับไตรกลีเซอไรด์ในเซรั่ม
||- s6 glu, ระดับน้ำตาลในเลือด|

> 🎓 จำไว้ว่า นี่คือการเรียนรู้แบบมีผู้สอน และเราต้องมีเป้าหมายที่ชื่อว่า 'y'

ก่อนที่คุณจะสามารถจัดการข้อมูลด้วย R ได้ คุณจำเป็นต้องนำเข้าข้อมูลเข้าสู่หน่วยความจำของ R หรือสร้างการเชื่อมต่อกับข้อมูลที่ R สามารถใช้เพื่อเข้าถึงข้อมูลจากระยะไกล

> แพ็กเกจ [readr](https://readr.tidyverse.org/) ซึ่งเป็นส่วนหนึ่งของ Tidyverse ให้วิธีการที่รวดเร็วและเป็นมิตรในการอ่านข้อมูลแบบสี่เหลี่ยมเข้าสู่ R

ตอนนี้ เรามาโหลดชุดข้อมูลโรคเบาหวานจาก URL แหล่งข้อมูลนี้: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

นอกจากนี้ เราจะทำการตรวจสอบความสมเหตุสมผลของข้อมูลโดยใช้ `glimpse()` และแสดง 5 แถวแรกโดยใช้ `slice()`

ก่อนที่จะไปต่อ เรามาแนะนำสิ่งที่คุณจะพบเจอบ่อยในโค้ด R 🥁🥁: ตัวดำเนินการ pipe `%>%`

ตัวดำเนินการ pipe (`%>%`) ทำการดำเนินการตามลำดับตรรกะโดยส่งวัตถุไปข้างหน้าเข้าสู่ฟังก์ชันหรือคำสั่ง คุณสามารถคิดว่าตัวดำเนินการ pipe เป็นการพูดว่า "และจากนั้น" ในโค้ดของคุณ


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` แสดงให้เห็นว่าข้อมูลนี้มี 442 แถว และ 11 คอลัมน์ โดยทุกคอลัมน์เป็นชนิดข้อมูล `double`

<br>

> `glimpse()` และ `slice()` เป็นฟังก์ชันใน [`dplyr`](https://dplyr.tidyverse.org/) Dplyr ซึ่งเป็นส่วนหนึ่งของ Tidyverse เป็นไวยากรณ์สำหรับการจัดการข้อมูลที่ให้ชุดคำกริยาที่สอดคล้องกันเพื่อช่วยแก้ปัญหาท้าทายที่พบบ่อยที่สุดในการจัดการข้อมูล

<br>

ตอนนี้เมื่อเรามีข้อมูลแล้ว เรามาโฟกัสไปที่คุณลักษณะหนึ่ง (`bmi`) เพื่อใช้ในแบบฝึกหัดนี้ ซึ่งจะต้องเลือกคอลัมน์ที่ต้องการ แล้วเราจะทำสิ่งนี้ได้อย่างไร?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) ช่วยให้เราสามารถ *เลือก* (และเปลี่ยนชื่อได้ถ้าต้องการ) คอลัมน์ใน data frame


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. ข้อมูลการฝึกและการทดสอบ

ในกระบวนการเรียนรู้แบบมีผู้สอน มักจะมีการ *แบ่ง* ข้อมูลออกเป็นสองชุดย่อย; ชุดที่ใหญ่กว่า (โดยทั่วไป) สำหรับการฝึกโมเดล และชุดที่เล็กกว่า "สำรองไว้" เพื่อดูว่าโมเดลทำงานได้ดีเพียงใด

ตอนนี้เรามีข้อมูลพร้อมแล้ว เราสามารถดูได้ว่ามีวิธีที่เครื่องสามารถช่วยกำหนดการแบ่งที่เหมาะสมระหว่างตัวเลขในชุดข้อมูลนี้หรือไม่ เราสามารถใช้แพ็กเกจ [rsample](https://tidymodels.github.io/rsample/) ซึ่งเป็นส่วนหนึ่งของเฟรมเวิร์ก Tidymodels เพื่อสร้างออบเจ็กต์ที่มีข้อมูลเกี่ยวกับ *วิธี* การแบ่งข้อมูล และใช้ฟังก์ชัน rsample อีกสองตัวเพื่อดึงชุดข้อมูลการฝึกและการทดสอบที่สร้างขึ้น:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. ฝึกโมเดลการถดถอยเชิงเส้นด้วย Tidymodels

ตอนนี้เราพร้อมที่จะฝึกโมเดลของเราแล้ว!

ใน Tidymodels คุณสามารถกำหนดโมเดลโดยใช้ `parsnip()` โดยระบุแนวคิดสามอย่างดังนี้:

-   **ประเภทของโมเดล** ใช้เพื่อแยกแยะโมเดล เช่น การถดถอยเชิงเส้น การถดถอยโลจิสติก โมเดลต้นไม้ตัดสินใจ และอื่นๆ

-   **โหมดของโมเดล** รวมถึงตัวเลือกทั่วไป เช่น การถดถอยและการจำแนกประเภท; โมเดลบางประเภทสามารถรองรับทั้งสองโหมดนี้ ในขณะที่บางประเภทมีเพียงโหมดเดียว

-   **เครื่องมือของโมเดล** คือเครื่องมือคำนวณที่จะใช้ในการปรับโมเดล โดยมักจะเป็นแพ็กเกจใน R เช่น **`"lm"`** หรือ **`"ranger"`**

ข้อมูลเกี่ยวกับการสร้างโมเดลนี้จะถูกบันทึกไว้ในสเปคโมเดล ดังนั้นเรามาสร้างกันเลย!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

หลังจากที่โมเดลถูก *กำหนด* แล้ว โมเดลสามารถถูก `ประเมิน` หรือ `ฝึกฝน` ได้โดยใช้ฟังก์ชัน [`fit()`](https://parsnip.tidymodels.org/reference/fit.html) ซึ่งมักจะใช้สูตรและข้อมูลบางส่วน

`y ~ .` หมายความว่าเราจะปรับ `y` ให้เป็นค่าที่คาดการณ์/เป้าหมาย โดยอธิบายด้วยตัวแปรทำนาย/คุณลักษณะทั้งหมด เช่น `.` (ในกรณีนี้ เรามีตัวแปรทำนายเพียงตัวเดียว: `bmi`)


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

จากผลลัพธ์ของโมเดล เราสามารถเห็นค่าสัมประสิทธิ์ที่ได้จากการฝึกฝน ซึ่งค่าสัมประสิทธิ์เหล่านี้แสดงถึงค่าของเส้นที่เหมาะสมที่สุดที่ช่วยลดข้อผิดพลาดโดยรวมระหว่างตัวแปรจริงและตัวแปรที่คาดการณ์

<br>

## 5. ทำนายผลบนชุดข้อมูลทดสอบ

เมื่อเราได้ฝึกฝนโมเดลแล้ว เราสามารถใช้มันเพื่อทำนายการพัฒนาของโรค y สำหรับชุดข้อมูลทดสอบโดยใช้ [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) ซึ่งจะถูกใช้เพื่อวาดเส้นแบ่งระหว่างกลุ่มข้อมูล


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

เย้! 💃🕺 เราเพิ่งฝึกโมเดลและใช้มันเพื่อทำการพยากรณ์!

เมื่อทำการพยากรณ์ ตามธรรมเนียมของ tidymodels จะสร้างผลลัพธ์ในรูปแบบ tibble หรือ data frame ที่มีชื่อคอลัมน์มาตรฐานเสมอ วิธีนี้ช่วยให้สามารถรวมข้อมูลต้นฉบับและผลการพยากรณ์ในรูปแบบที่ใช้งานได้ง่ายสำหรับการดำเนินการต่อ เช่น การสร้างกราฟ

`dplyr::bind_cols()` ช่วยรวมหลาย data frame เข้าด้วยกันอย่างมีประสิทธิภาพในรูปแบบคอลัมน์


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. แสดงผลการสร้างโมเดล

ถึงเวลาที่จะดูผลลัพธ์ในรูปแบบภาพแล้ว 📈 เราจะสร้างกราฟกระจายของค่าทั้งหมดใน `y` และ `bmi` จากชุดข้อมูลทดสอบ จากนั้นใช้ค่าที่คาดการณ์เพื่อวาดเส้นในตำแหน่งที่เหมาะสมที่สุด ระหว่างกลุ่มข้อมูลของโมเดล

R มีระบบหลายแบบสำหรับการสร้างกราฟ แต่ `ggplot2` เป็นหนึ่งในระบบที่ดูเรียบง่ายและมีความหลากหลายมากที่สุด ระบบนี้ช่วยให้คุณสามารถสร้างกราฟโดย **การรวมองค์ประกอบที่เป็นอิสระเข้าด้วยกัน**


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ ลองคิดดูว่าเกิดอะไรขึ้นที่นี่ เส้นตรงเส้นหนึ่งกำลังผ่านจุดข้อมูลเล็กๆ หลายจุด แต่จริงๆ แล้วมันกำลังทำอะไรอยู่? คุณเห็นไหมว่าคุณควรจะสามารถใช้เส้นนี้เพื่อทำนายว่าจุดข้อมูลใหม่ที่ยังไม่เคยเห็นควรจะอยู่ตรงไหนในความสัมพันธ์กับแกน y ของกราฟ? ลองอธิบายเป็นคำพูดถึงการใช้งานจริงของโมเดลนี้

ขอแสดงความยินดี! คุณสร้างโมเดลการถดถอยเชิงเส้นตัวแรกของคุณ สร้างการทำนายด้วยมัน และแสดงผลในกราฟ!



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่แม่นยำ เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลภาษาจากผู้เชี่ยวชาญ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่ผิดพลาดซึ่งเกิดจากการใช้การแปลนี้
